In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [77]:
# 主要数据
df_train_origin = pd.read_csv('data/application_train.csv')
df_test_origin = pd.read_csv('data/application_test.csv')

combine_origin = [df_train_origin,df_test_origin]

In [ ]:
# TARGET：1-会逾期；2-其他情况
# NAME_CONTRACT_TYPE（合同类型-贷款方式）：Cash loans（现金贷款）；Revolving loans（循环贷款）
# CODE_GENDER（客户性别）：F（女）；M（男）；XNA（）
# FLAG_OWN_CAR（是否拥有车辆）：N/Y
# FLAG_OWN_REALTY（是否拥有房产）：N/Y
# CNT_CHILDREN（拥有子女数量）：int
# AMT_INCOME_TOTAL（收入）：
# AMT_CREDIT（贷款的信贷金额）：
# AMT_ANNUITY（年金-每期还款）
# AMT_GOODS_PRICE（商品价格）

# NAME_TYPE_SUITE（客户申请贷款时由谁陪同）：1.Unaccompanied（无陪同）；2.Family；3.Spouse, partner（配偶伴侣）；4.Children；5.Other_B；6.Other_A；7.Group of people
# NAME_INCOME_TYPE（收入类型）：1.Working；2.Commercial associate（商业伙伴）；3.Pensioner（养老金）；4.State servant（）；5.Unemployed（失业）；6.Student；7.Businessman；8.Maternity leave
# NAME_EDUCATION_TYPE（受教育程度）：1.Secondary / secondary special（高中）；2.Higher education（大学以上学历）；3.Incomplete higher（大学在读/未完成大学）；4.Lower secondary（初中）；5.Academic degree（学位）
# NAME_FAMILY_STATUS（家庭情况）：1.Married；2.Single / not married；3.Civil marriage；4.Separated；5.Widow；6.Unknown
# NAME_HOUSING_TYPE（住房情况）：1.House / apartment；2.With parents；3.Municipal apartment；4.Rented apartment；5.Office apartment；6.Co-op apartment
# REGION_POPULATION_RELATIVE（数量越高表示客户所在区域人口越多）：
# DAYS_BIRTH（年龄-天）：
# DAYS_EMPLOYED（当前的工作持续时间-天）：
# DAYS_LAST_PHONE_CHANGE（-天）：
# DAYS_REGISTRATION（客户在申请前几天更改了注册-天）：
# DAYS_ID_PUBLISH（客户在申请前多少天更改了申请贷款的身份证明文件-天）：
# OWN_CAR_AGE（车的年龄）：1.NaN；2.int
# FLAG_MOBIL（是否预留的手机号）：1/0
# FLAG_EMP_PHONE：1/0
# FLAG_WORK_PHONE：1/0
# FLAG_CONT_MOBILE：1/0
# FLAG_PHONE：1/0
# FLAG_EMAIL：1/0
# FLAG_DOCUMENT_2（是否提供了文件2~21）：1/0
# OCCUPATION_TYPE（职业）：1.Laborers；2.Sales staff；3.Core staff；4.Managers；5.Drivers；6.High skill tech staff；7.Accountants；8.Medicine staff；9.Security staff；10.Cooking staff；
#                          11.Cleaning staff；12.Private service staff；13.Low-skill Laborers；14.Waiters/barmen staff；15.Secretaries；16.Realty agents；17.HR staff；18.IT staff
# ORGANIZATION_TYPE（工作的组织类型）：类型比较多
# CNT_FAM_MEMBERS（家庭成员数量）：int
# REGION_RATING_CLIENT（我们对客户所在地区的评级）：1/2/3
# REGION_RATING_CLIENT_W_CITY（考虑城市因素，我们对客户所在地区的评级）：1/2/3
# WEEKDAY_APPR_PROCESS_START（申请贷款的在一周中的哪一天）：1.SUNDAY；2.SATURDAY；3.FRIDAY；4.THURSDAY；5.MONDAY；6.WEDNESDAY；7.TUESDAY
# HOUR_APPR_PROCESS_START（申请贷款的在一周中的几点）：0~23
# REG_REGION_NOT_LIVE_REGION（永久地址与联系地址是否在同一区域级别）：0（相同）；1（不同）
# REG_REGION_NOT_WORK_REGION（永久地址与工作地址是否在同一区域级别）：0（相同）；1（不同）
# LIVE_REGION_NOT_WORK_REGION（联系地址与工作地址是否在同一区域级别）：0（相同）；1（不同）
# REG_CITY_NOT_LIVE_CITY（永久地址与联系地址是否在同一城市级别）：0/1
# REG_CITY_NOT_WORK_CITY：0/1
# LIVE_CITY_NOT_WORK_CITY：0/1
# EXT_SOURCE_1（来自外部数据源的标准化评分）：0~1
# EXT_SOURCE_2
# EXT_SOURCE_3

## 客户居住的建筑的标准化信息，平均(_AVG后缀)，modus后缀(_MODE后缀)，中值(_MEDI后缀)  0~1 
## 公寓大小，公共面积，居住面积，建筑年龄，电梯数量，入口数量，建筑状态，楼层数
# APARTMENTS_AVG（公寓大小）：1.NaN；2.0~1
# BASEMENTAREA_AVG（地下室）：1.NaN；2.0~1
# YEARS_BEGINEXPLUATATION_AVG：1.NaN；2.0~1
# YEARS_BUILD_AVG：1.NaN；2.0~1
# COMMONAREA_AVG：1.NaN；2.0~1
# ELEVATORS_AVG：1.NaN；2.0~1
# ENTRANCES_AVG：1.NaN；2.0~1
# FLOORSMAX_AVG：1.NaN；2.0~1
# FLOORSMIN_AVG：1.NaN；2.0~1
# LANDAREA_AVG：1.NaN；2.0~1
# LIVINGAPARTMENTS_AVG：1.NaN；2.0~1
# LIVINGAREA_AVG：1.NaN；2.0~1
# NONLIVINGAPARTMENTS_AVG：1.NaN；2.0~1
# NONLIVINGAREA_AVG：1.NaN；2.0~1

# OBS_30_CNT_SOCIAL_CIRCLE（逾期30天，对客户社交环境的观察有多少次）：
# DEF_30_CNT_SOCIAL_CIRCLE（逾期30天的客户社会环境观察值）
# OBS_60_CNT_SOCIAL_CIRCLE
# DEF_60_CNT_SOCIAL_CIRCLE

# AMT_REQ_CREDIT_BUREAU_HOUR（在申请前一小时向信用局询问客户的数量）
# AMT_REQ_CREDIT_BUREAU_DAY（在申请前一天(不包括申请前一小时)向信贷局查询客户的数目）
# AMT_REQ_CREDIT_BUREAU_WEEK
# AMT_REQ_CREDIT_BUREAU_MON
# AMT_REQ_CREDIT_BUREAU_QRT
# AMT_REQ_CREDIT_BUREAU_YEAR

In [74]:
# 由其他金融机构提供给客户的所有以前的信贷，这些信贷已报告给信贷局（针对在我们的样本中有贷款的客户）。
# 对于我们样本中的每笔贷款，行数与客户在申请日期之前在信用局中拥有的信用数一样多。
df_bureau = pd.read_csv('data/bureau.csv')

In [ ]:
# CREDIT_ACTIVE（信用局(CB)报告的信用的状态）：1.Closed；2.Active；3.Sold；4.Bad debt
# CREDIT_CURRENCY（由资信局重新编码的货币）：currency 1~4
# DAYS_CREDIT（客户在当前申请前多少天申请了信用局的信用-天）：
# CREDIT_DAY_OVERDUE（在我们的样本中，在申请相关贷款时CB信用逾期的天数-天）：
# DAYS_CREDIT_ENDDATE（在家庭信贷申请时剩余的CB信贷期限-天）：
# DAYS_ENDDATE_FACT（在Home credit申请时CB信用结束的天数(仅适用于关闭信用)）：
# AMT_CREDIT_MAX_OVERDUE（到目前为止征信局的最高逾期金额(在我们样本的贷款申请日期)）：
# CNT_CREDIT_PROLONG（信用局的信用延长了多少次）：
# AMT_CREDIT_SUM（当前信贷金额为信用局的信贷）：
# AMT_CREDIT_SUM_DEBT（当前债务的信用局信用）
# AMT_CREDIT_SUM_LIMIT（在资信局报告的信用卡当前信用额度）
# AMT_CREDIT_SUM_OVERDUE（在资信局的信贷上的当前逾期金额）
# CREDIT_TYPE（信用局信用类型）：类型比较多
# DAYS_CREDIT_UPDATE（在申请贷款前多少天，资信局的最后资信信息来了）
# AMT_ANNUITY（年金）

In [85]:
# 信用局中以前信用的每月余额。
# 该表在向信用局报告的每个先前信用的历史记录的每个月中都有一行–即该表具有（样本中的＃贷款*相对先前信用的数量*我们可以观察到先前信用的历史的月份数）行。
df_bureau_balance = pd.read_csv('data/bureau_balance.csv')

In [88]:
# 申请人通过房屋信贷拥有的先前信用卡的月度余额快照。
# 该表格在与样本中的贷款相关的房屋信贷（消费者信贷和现金贷款）中的每个先前信用的历史记录的每个月都有一行-即该表具有（（样本中的贷款*相对以前的信用卡数量*的＃个）以前的信用卡行中有一些历史记录的月份。
df_credit_card_balance = pd.read_csv('data/credit_card_balance.csv')

In [91]:
# 申请人通过房屋信贷拥有的先前POS（销售点）和现金贷款的每月余额快照。
# 该表在与样本中的贷款相关的房屋信贷（消费者信贷和现金贷款）中的每个先前信贷的历史记录的每个月中都有一行-即该表具有（（样本中的贷款*相对先前信贷的数量*月数）在其中，我们有一些历史记录可用于查看以前的信用记录。
df_POS_CASH_balance = pd.read_csv('data/POS_CASH_balance.csv')

In [94]:
# 在我们的样本中有贷款的客户以前所有的房屋信贷申请。
# 在我们的数据样本中，每个与贷款相关的先前申请都有一行。
df_previous_application = pd.read_csv('data/previous_application.csv')

In [ ]:
# 与我们样本中的贷款相关的房屋信贷中先前已支付的信贷的还款历史。
# 有a）每笔付款都有一行，另加b）每笔未付款都有一行。
# 一行等于我们分期付款中一笔还清一笔贷款，或者相当于一笔分期付款，相当于一笔以前一笔与贷款相关的房屋信用信贷的付款。
df_installments_payments = pd.read_csv('data/installments_payments.csv')

In [4]:
df_train_origin.select_dtypes('object').apply(pd.Series.nunique).sort_values()

NAME_CONTRACT_TYPE             2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
EMERGENCYSTATE_MODE            2
CODE_GENDER                    3
HOUSETYPE_MODE                 3
FONDKAPREMONT_MODE             4
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
NAME_TYPE_SUITE                7
WEEKDAY_APPR_PROCESS_START     7
WALLSMATERIAL_MODE             7
NAME_INCOME_TYPE               8
OCCUPATION_TYPE               18
ORGANIZATION_TYPE             58
dtype: int64

In [154]:
# df_train[["FLAG_OWN_CAR", "TARGET"]].groupby(['FLAG_OWN_CAR'], as_index=False).mean().sort_values(by='TARGET', ascending=False)

# 收教育程度高低和逾期存在关系
# df_train_origin[["NAME_EDUCATION_TYPE", "TARGET"]].groupby(['NAME_EDUCATION_TYPE'], as_index=False).mean().sort_values(by='TARGET', ascending=False)

# 通过箱型图观察，每期还款金额会影响逾期
# df_train_origin[df_train_origin['TARGET'] == 1][['AMT_ANNUITY']].plot.box()
# df_train_origin[df_train_origin['TARGET'] == 0][['AMT_ANNUITY']].plot.box()

LabelEncoder()

0    0
1    0
2    1
3    0
4    0
Name: NAME_CONTRACT_TYPE, dtype: int32

In [78]:
combine = []
le = preprocessing.LabelEncoder()
ohe = preprocessing.OneHotEncoder(categories='auto')
for i,dataset in enumerate(combine_origin):
    
    df = pd.DataFrame()
    try:
        df = dataset[['SK_ID_CURR','TARGET']]
    except:
        df = dataset[['SK_ID_CURR']]

    dataset.replace(np.nan, 0, inplace=True) 
    dataset.replace(np.inf, 0, inplace=True)
    
    dataset['AMT_INCOME_TOTAL'] = dataset['AMT_INCOME_TOTAL'].apply(lambda x:round(x / 10000,2))
    dataset['AMT_CREDIT'] = dataset['AMT_CREDIT'].apply(lambda x:round(x / 10000,2))
    dataset['AMT_ANNUITY'] = dataset['AMT_ANNUITY'].apply(lambda x:round(x / 10000,2))
    dataset['AMT_GOODS_PRICE'] = dataset['AMT_GOODS_PRICE'].apply(lambda x:round(x / 10000,2))
    
    dataset['REGION_POPULATION_RELATIVE'] = dataset['REGION_POPULATION_RELATIVE'].apply(lambda x:round(x,3))
    dataset['EXT_SOURCE_1'] = dataset['EXT_SOURCE_1'].apply(lambda x:round(x,3))
    dataset['EXT_SOURCE_2'] = dataset['EXT_SOURCE_2'].apply(lambda x:round(x,3))
    dataset['EXT_SOURCE_3'] = dataset['EXT_SOURCE_3'].apply(lambda x:round(x,3))
    dataset['DAYS_BIRTH'] = dataset['DAYS_BIRTH'].apply(lambda x:round(x/-365,1))
    dataset['DAYS_EMPLOYED'] = dataset['DAYS_EMPLOYED'].apply(lambda x:round(x/-365,1))
    dataset['DAYS_REGISTRATION'] = dataset['DAYS_REGISTRATION'].apply(lambda x:round(x/-365,1))
    dataset['DAYS_ID_PUBLISH'] = dataset['DAYS_ID_PUBLISH'].apply(lambda x:round(x/-365,1))
    dataset['DAYS_LAST_PHONE_CHANGE'] = dataset['DAYS_LAST_PHONE_CHANGE'].apply(lambda x:round(x/-365,1))
    
    dataset['NAME_CONTRACT_TYPE'] = le.fit_transform(dataset['NAME_CONTRACT_TYPE'])
#     dataset['NAME_CONTRACT_TYPE'] = dataset['NAME_CONTRACT_TYPE'].map({"Cash loans":1,"Revolving loans":2})
    
    dataset['FLAG_OWN_CAR'] = le.fit_transform(dataset['FLAG_OWN_CAR'])
#     dataset['FLAG_OWN_CAR'] = dataset['FLAG_OWN_CAR'].map({"N":0,"Y":1})
    dataset['FLAG_OWN_REALTY'] = le.transform(dataset['FLAG_OWN_REALTY'])
#     dataset['FLAG_OWN_REALTY'] = dataset['FLAG_OWN_REALTY'].map({"N":0,"Y":1})
    
#     dataset['CODE_GENDER'] = dataset['CODE_GENDER'].map({"F":1,"M":2,"XNA":3})
    
#     dataset['NAME_TYPE_SUITE'] = le.fit_transform(dataset['NAME_TYPE_SUITE'])
#     dataset['NAME_TYPE_SUITE'] = dataset['NAME_TYPE_SUITE'].map({"Unaccompanied":1,"Family":2,"Spouse, partner":3,"Children":4,"Other_B":5,"Other_A":6,"Group of people":7})
#     dataset['NAME_TYPE_SUITE'] = dataset['NAME_TYPE_SUITE'].fillna(0)

#     dataset['NAME_INCOME_TYPE'] = le.fit_transform(dataset['NAME_INCOME_TYPE'])
#     dataset['NAME_INCOME_TYPE'] = dataset['NAME_INCOME_TYPE'].map({"Working":1,"Commercial associate":2,"Pensioner":3,"State servant":4,"Unemployed":5,"Student":6,"Businessman":7,"Maternity leave":8})

#     dataset['NAME_FAMILY_STATUS'] = le.fit_transform(dataset['NAME_FAMILY_STATUS'])
#     dataset['NAME_FAMILY_STATUS'] = dataset['NAME_FAMILY_STATUS'].map({"Married":1,"Single / not married":2,"Civil marriage":3,"Separated":4,"Widow":5,"Unknown":6})

#     dataset['NAME_HOUSING_TYPE'] = le.fit_transform(dataset['NAME_HOUSING_TYPE'])
#     dataset['NAME_HOUSING_TYPE'] = dataset['NAME_HOUSING_TYPE'].map({"House / apartment":1,"With parents":2,"Municipal apartment":3,"Rented apartment":4,"Office apartment":5,"Co-op apartment":6})
    
#     dataset['WEEKDAY_APPR_PROCESS_START'] = dataset['WEEKDAY_APPR_PROCESS_START'].map({"SUNDAY":1,"SATURDAY":2,"FRIDAY":3,"THURSDAY":4,"MONDAY":5,"WEDNESDAY":6,"TUESDAY":7})
#     dataset['NAME_EDUCATION_TYPE'] = dataset['NAME_EDUCATION_TYPE'].map({"Lower secondary":1,"Secondary / secondary special":2,"Incomplete higher":3,"Higher education":4,"Academic degree":5})
    
    for suffix in ['_AVG','_MODE','_MEDI']:
        dataset = dataset.drop(['APARTMENTS' + suffix], axis=1)
        dataset = dataset.drop(['BASEMENTAREA' + suffix], axis=1)
        dataset = dataset.drop(['YEARS_BEGINEXPLUATATION' + suffix], axis=1)
        dataset = dataset.drop(['YEARS_BUILD' + suffix], axis=1)
        dataset = dataset.drop(['COMMONAREA' + suffix], axis=1)
        dataset = dataset.drop(['ELEVATORS' + suffix], axis=1)
        dataset = dataset.drop(['ENTRANCES' + suffix], axis=1)
        dataset = dataset.drop(['FLOORSMAX' + suffix], axis=1)
        dataset = dataset.drop(['FLOORSMIN' + suffix], axis=1)
        dataset = dataset.drop(['LANDAREA' + suffix], axis=1)
        dataset = dataset.drop(['LIVINGAPARTMENTS' + suffix], axis=1)
        dataset = dataset.drop(['LIVINGAREA' + suffix], axis=1)
        dataset = dataset.drop(['NONLIVINGAPARTMENTS' + suffix], axis=1)
        dataset = dataset.drop(['NONLIVINGAREA' + suffix], axis=1)
        
    dataset = dataset.drop(['FONDKAPREMONT_MODE'], axis=1)
    dataset = dataset.drop(['HOUSETYPE_MODE'], axis=1)
    dataset = dataset.drop(['TOTALAREA_MODE'], axis=1)
    dataset = dataset.drop(['WALLSMATERIAL_MODE'], axis=1)
    dataset = dataset.drop(['EMERGENCYSTATE_MODE'], axis=1)
    
    #商品价格-填充空值
    dataset.loc[dataset['AMT_GOODS_PRICE'] == 0,'AMT_GOODS_PRICE'] = dataset['AMT_GOODS_PRICE'].median()
    #每期还款-填充空值
    dataset.loc[dataset['AMT_ANNUITY'] == 0,'AMT_ANNUITY'] = dataset['AMT_ANNUITY'].median()
    
    dataset = pd.get_dummies(dataset)
    
    combine_origin[i] = dataset
    
    # 收入/信贷金额
    dataset['RATE_INCOME_CREDIT'] = dataset['AMT_INCOME_TOTAL'] / dataset['AMT_CREDIT']
    # 收入/商品价格
    dataset['RATE_INCOME_GPRICE'] = dataset['AMT_INCOME_TOTAL'] / dataset['AMT_GOODS_PRICE']
    # 每期还款/信贷金额
    dataset['RATE_ANNUITY_CREDIT'] = dataset['AMT_ANNUITY'] / dataset['AMT_CREDIT']
    # 每期还款/收入
    dataset['RATE_ANNUITY_INCOME'] = dataset['AMT_ANNUITY'] / dataset['AMT_INCOME_TOTAL']
    
    df = pd.concat([df,dataset[['RATE_INCOME_CREDIT','RATE_INCOME_GPRICE','RATE_ANNUITY_CREDIT','RATE_ANNUITY_INCOME',
     'NAME_CONTRACT_TYPE',
     'FLAG_OWN_CAR',
     'CNT_CHILDREN',
     'AMT_CREDIT',
     'AMT_ANNUITY',
     'AMT_GOODS_PRICE',
     'REGION_POPULATION_RELATIVE',
     'DAYS_BIRTH',
     'DAYS_EMPLOYED',
     'DAYS_REGISTRATION',
     'DAYS_ID_PUBLISH',
     'FLAG_EMP_PHONE',
     'FLAG_WORK_PHONE',
     'FLAG_PHONE',
     'CNT_FAM_MEMBERS',
     'REGION_RATING_CLIENT',
     'REGION_RATING_CLIENT_W_CITY',
     'HOUR_APPR_PROCESS_START',
     'REG_REGION_NOT_WORK_REGION',
     'REG_CITY_NOT_LIVE_CITY',
     'REG_CITY_NOT_WORK_CITY',
     'LIVE_CITY_NOT_WORK_CITY',
     'EXT_SOURCE_1',
     'EXT_SOURCE_2',
     'EXT_SOURCE_3',
     'OBS_30_CNT_SOCIAL_CIRCLE',
     'DEF_30_CNT_SOCIAL_CIRCLE',
     'OBS_60_CNT_SOCIAL_CIRCLE',
     'DEF_60_CNT_SOCIAL_CIRCLE',
     'DAYS_LAST_PHONE_CHANGE',
     'FLAG_DOCUMENT_3',
     'FLAG_DOCUMENT_6',
     'FLAG_DOCUMENT_8',
     'FLAG_DOCUMENT_13',
     'FLAG_DOCUMENT_14',
     'FLAG_DOCUMENT_15',
     'FLAG_DOCUMENT_16',
     'FLAG_DOCUMENT_18',
     'AMT_REQ_CREDIT_BUREAU_MON',
     'CODE_GENDER_F',
     'CODE_GENDER_M',
     'NAME_TYPE_SUITE_Family',
     'NAME_TYPE_SUITE_Unaccompanied',
     'NAME_INCOME_TYPE_Commercial associate',
     'NAME_INCOME_TYPE_Pensioner',
     'NAME_INCOME_TYPE_State servant',
     'NAME_INCOME_TYPE_Unemployed',
     'NAME_INCOME_TYPE_Working',
     'NAME_EDUCATION_TYPE_Higher education',
     'NAME_EDUCATION_TYPE_Lower secondary',
     'NAME_EDUCATION_TYPE_Secondary / secondary special',
     'NAME_FAMILY_STATUS_Civil marriage',
     'NAME_FAMILY_STATUS_Married',
     'NAME_FAMILY_STATUS_Single / not married',
     'NAME_FAMILY_STATUS_Widow',
     'NAME_HOUSING_TYPE_House / apartment',
     'NAME_HOUSING_TYPE_Rented apartment',
     'NAME_HOUSING_TYPE_With parents',
     'OCCUPATION_TYPE_0',
     'OCCUPATION_TYPE_Accountants',
     'OCCUPATION_TYPE_Cleaning staff',
     'OCCUPATION_TYPE_Cooking staff',
     'OCCUPATION_TYPE_Core staff',
     'OCCUPATION_TYPE_Drivers',
     'OCCUPATION_TYPE_High skill tech staff',
     'OCCUPATION_TYPE_Laborers',
     'OCCUPATION_TYPE_Low-skill Laborers',
     'OCCUPATION_TYPE_Managers',
     'OCCUPATION_TYPE_Medicine staff',
     'OCCUPATION_TYPE_Sales staff',
     'OCCUPATION_TYPE_Security staff',
     'OCCUPATION_TYPE_Waiters/barmen staff',
     'ORGANIZATION_TYPE_Agriculture',
     'ORGANIZATION_TYPE_Bank',
     'ORGANIZATION_TYPE_Business Entity Type 3',
     'ORGANIZATION_TYPE_Construction',
     'ORGANIZATION_TYPE_Government',
     'ORGANIZATION_TYPE_Industry: type 1',
     'ORGANIZATION_TYPE_Industry: type 3',
     'ORGANIZATION_TYPE_Medicine',
     'ORGANIZATION_TYPE_Military',
     'ORGANIZATION_TYPE_Police',
     'ORGANIZATION_TYPE_Restaurant',
     'ORGANIZATION_TYPE_School',
     'ORGANIZATION_TYPE_Security',
     'ORGANIZATION_TYPE_Security Ministries',
     'ORGANIZATION_TYPE_Self-employed',
     'ORGANIZATION_TYPE_Trade: type 3',
     'ORGANIZATION_TYPE_Trade: type 7',
     'ORGANIZATION_TYPE_Transport: type 3',
     'ORGANIZATION_TYPE_University',
     'ORGANIZATION_TYPE_XNA',
     'RATE_ANNUITY_CREDIT',
     'RATE_ANNUITY_INCOME']]],axis=1)
    
    combine.append(df)


In [82]:
combine[0].head()

,SK_ID_CURR,TARGET,RATE_INCOME_CREDIT,RATE_INCOME_GPRICE,RATE_ANNUITY_CREDIT,RATE_ANNUITY_INCOME,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,CNT_CHILDREN,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_18,AMT_REQ_CREDIT_BUREAU_MON,CODE_GENDER_F,CODE_GENDER_M,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_0,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,RATE_ANNUITY_CREDIT,RATE_ANNUITY_INCOME
0,100002,1,0.498032,0.576923,0.060748,0.121975,0,0,0,40.66,2.47,35.10,0.019,25.9,1.7,10.0,5.8,1,0,1,1.0,2,2,10,0,0,0,0,0.083,0.263,0.139,2.0,2.0,2.0,2.0,3.1,1,0,0,0,0,0,0,0,0.0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.060748,0.121975
1,100003,0,0.208736,0.239044,0.027600,0.132222,0,0,0,129.35,3.57,112.95,0.004,45.9,3.3,3.2,0.8,1,0,1,2.0,1,1,11,0,0,0,0,0.311,0.622,0.000,1.0,0.0,1.0,0.0,2.3,1,0,0,0,0,0,0,0,0.0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.027600,0.132222
2,100004,0,0.500000,0.500000,0.050370,0.100741,1,1,0,13.50,0.68,13.50,0.010,52.2,0.6,11.7,6.9,1,1,1,1.0,2,2,9,0,0,0,0,0.000,0.556,0.730,0.0,0.0,0.0,0.0,2.2,0,0,0,0,0,0,0,0,0.0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.050370,0.100741
3,100006,0,0.431724,0.454545,0.094979,0.220000,0,0,0,31.27,2.97,29.70,0.008,52.1,8.3,26.9,6.7,1,0,0,2.0,2,2,17,0,0,0,0,0.000,0.650,0.000,2.0,0.0,2.0,0.0,1.7,1,0,0,0,0,0,0,0,0.0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.094979,0.220000
4,100007,0,0.236842,0.236842,0.042690,0.180247,0,0,0,51.30,2.19,51.30,0.029,54.6,8.3,11.8,9.5,1,0,0,1.0,2,2,11,0,0,1,1,0.000,0.323,0.000,0.0,0.0,0.0,0.0,3.0,0,0,1,0,0,0,0,0,0.0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.042690,0.180247


In [51]:
df_train_origin['NAME_HOUSING_TYPE'].value_counts()

1.0    272868
0.0     34643
Name: NAME_HOUSING_TYPE, dtype: int64

In [80]:
df_train = combine[0]
df_test = combine[1]

X_train = df_train.drop(["TARGET",'SK_ID_CURR'], axis=1)
Y_train = df_train["TARGET"]

X_test  = df_test.drop("SK_ID_CURR", axis=1).copy()

In [75]:
# 选择特性

select = SelectPercentile(percentile=50)
select.fit(X_train, Y_train)

X_train_selected = select.transform(X_train)
support = select.get_support()

support_list = []
colunms = X_train.columns.values
for i,s in enumerate(support):
    if s:
        support_list.append(colunms[i])

In [81]:
# 特征缩放
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [45]:
# model = linear_model.LinearRegression()
# model.fit(X_train, Y_train)
# Y_pred = model.predict(X_test)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [83]:
logreg = LogisticRegression(solver='liblinear',max_iter=10000)
logreg.fit(X_train,Y_train)
# Y_pred = logreg.predict(X_test)
Y_pred = logreg.predict_proba(X_test)
# acc_log = logreg.score(X_train,Y_train)

In [84]:
pred = []
for t in Y_pred:
    pred.append(t[1])

In [85]:
max(pred)

0.9673548077219835

In [86]:
submission = pd.DataFrame({
        "SK_ID_CURR": df_test_origin["SK_ID_CURR"],
        "TARGET": pred
    })

In [87]:
submission.to_csv('result/submission.csv', index=False)